# Assignment 9: Improved Nim Q-Learning

## Solution Overview

This notebook implements an enhanced Q-learning algorithm for the Nim game with three key improvements:

1. **Loss Penalty**: Adds a -100 penalty for losing games (original only rewarded wins)
2. **Self-Play Training**: Trains the Q-learner against itself and mixed opponents (Random, Guru) for better exploration
3. **Intermediate Rewards**: Provides bonus rewards for achieving favorable nim-sum positions
4. **Experience Replay**: Uses a buffer to replay past experiences for more stable learning
5. **Better Exploration Strategy**: Uses epsilon-greedy with decay for balanced exploration/exploitation

In [32]:
import numpy as np
from random import randint, choice
import random
from collections import defaultdict

# The number of piles is 3
# max number of items per pile
ITEMS_MX = 10

## Core Game Functions

In [33]:
# Initialize starting position
def init_game()->list:
    return [randint(1,ITEMS_MX), randint(1,ITEMS_MX), randint(1,ITEMS_MX)]

# Based on X-oring the item counts in piles - mathematical solution
def nim_guru(_st:list)->(int,int):
    xored = _st[0] ^ _st[1] ^ _st[2]
    if xored == 0:
        return nim_random(_st)
    for pile in range(3):
        s = _st[pile] ^ xored
        if s <= _st[pile]:
            return _st[pile]-s, pile

# Random Nim player
def nim_random(_st:list)->(int,int):
    pile = choice([i for i in range(3) if _st[i]>0])  # find the non-empty piles
    return randint(1, _st[pile]), pile  # random move

# Calculate nim-sum for intermediate rewards (NEW)
def calculate_nim_value(state):
    """Calculate the nim-sum (XOR) of the state"""
    return state[0] ^ state[1] ^ state[2]

## Original Q-Learner Implementation

In [34]:
def nim_qlearner_original(_st:list)->(int,int):
    global qtable
    # pick the best rewarding move, equation 1
    a = np.argmax(qtable[_st[0], _st[1], _st[2]])  # exploitation
    # index is based on move, pile
    move, pile = a%ITEMS_MX+1, a//ITEMS_MX
    # check if qtable has generated a random but game illegal move
    if move <= 0 or _st[pile] < move:
        move, pile = nim_random(_st)  # exploration
    return move, pile  # action

# Global Q-tables
qtable = None  # Original
qtable_improved = None  # Improved

# Original parameters
Alpha = 1.0
Gamma = 0.8
Reward = 100.0

# Equation 3 - update the qtable
def qtable_update(r:float, _st1:list, move:int, pile:int, q_future_best:float):
    global qtable
    a = pile*ITEMS_MX+move-1
    qtable[_st1[0], _st1[1], _st1[2], a] = Alpha * (r + Gamma * q_future_best)

# learn from _n games, randomly played to explore the possible states
def nim_qlearn(_n:int):
    global qtable
    # based on max items per pile
    qtable = np.zeros((ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX*3), dtype=np.float32)
    # play _n games
    for _ in range(_n):
        # first state is starting position
        st1 = init_game()
        while True:  # while game not finished
            # make a random move - exploration
            move, pile = nim_random(st1)
            st2 = list(st1)
            # make the move
            st2[pile] -= move  # --> last move I made
            if st2 == [0, 0, 0]:  # game ends
                qtable_update(Reward, st1, move, pile, 0)  # I won
                break  # new game

            qtable_update(0, st1, move, pile, np.max(qtable[st2[0], st2[1], st2[2]]))
            
            # Switch sides for play and learning
            st1 = st2

## IMPROVED Q-Learner Implementation

### Key Improvements:
1. **Epsilon-greedy exploration** with decay
2. **Negative rewards for losses** (-100 penalty)
3. **Self-play training** for better strategy discovery
4. **Intermediate rewards** based on nim-sum
5. **Experience replay buffer** for stable learning

In [35]:
# Improved Q-learner with epsilon-greedy
def nim_qlearner_improved(_st:list, epsilon=0.0)->(int,int):
    global qtable_improved
    
    # Epsilon-greedy exploration
    if np.random.random() < epsilon:
        return nim_random(_st)
    
    # Get Q-values for current state
    q_values = qtable_improved[_st[0], _st[1], _st[2]]
    
    # Find valid actions only
    valid_actions = []
    for a in range(ITEMS_MX * 3):
        move, pile = a%ITEMS_MX+1, a//ITEMS_MX
        if pile < 3 and 0 < move <= _st[pile]:
            valid_actions.append(a)
    
    if not valid_actions:
        return nim_random(_st)
    
    # Select best valid action
    valid_q_values = [q_values[a] for a in valid_actions]
    best_action_idx = valid_actions[np.argmax(valid_q_values)]
    
    move, pile = best_action_idx%ITEMS_MX+1, best_action_idx//ITEMS_MX
    return move, pile

# Improved parameters
Alpha_imp = 0.3  # Lower learning rate for stability
Gamma_imp = 0.95  # Higher discount for better long-term planning
Win_Reward = 100.0
Loss_Penalty = -100.0  # NEW: Penalty for losing
Nim_Sum_Bonus = 10.0  # NEW: Bonus for achieving nim-sum of 0

In [36]:
def nim_qlearn_improved(_n:int):
    """
    Improved Q-learning with:
    1. Self-play for better exploration
    2. Intermediate rewards based on nim-sum
    3. Penalty for losses
    4. Experience replay buffer
    5. Training against mixed opponents
    """
    global qtable_improved
    
    # Initialize Q-table with small random values to break ties
    qtable_improved = np.random.uniform(-0.01, 0.01, 
        (ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX*3)).astype(np.float32)
    
    # Experience replay buffer
    experience_buffer = []
    buffer_size = 1000
    batch_size = 32
    
    # Dynamic epsilon for exploration
    epsilon = 0.3  # Start with higher exploration
    
    # Mix of training opponents
    opponents = ['self', 'random', 'guru']
    
    for episode in range(_n):
        # Decay epsilon
        if episode % 500 == 0 and episode > 0:
            epsilon = max(0.05, epsilon * 0.95)
        
        # Choose training opponent
        opponent = opponents[episode % 3]
        
        state = init_game()
        game_history = []
        
        while True:
            # Q-learner's turn
            current_state = list(state)
            move, pile = nim_qlearner_improved(current_state, epsilon)
            action_idx = pile * ITEMS_MX + move - 1
            
            # Apply move
            state[pile] -= move
            next_state = list(state)
            
            # Calculate intermediate reward based on nim-sum
            nim_sum = calculate_nim_value(next_state)
            intermediate_reward = Nim_Sum_Bonus if nim_sum == 0 else 0
            
            # Check if game ended
            if next_state == [0, 0, 0]:
                # Q-learner wins
                reward = Win_Reward + intermediate_reward
                game_history.append((current_state, action_idx, next_state, reward, True))
                break
            else:
                game_history.append((current_state, action_idx, next_state, intermediate_reward, False))
            
            # Opponent's turn
            if opponent == 'self':
                move, pile = nim_qlearner_improved(state, epsilon)
            elif opponent == 'random':
                move, pile = nim_random(state)
            else:  # guru
                move, pile = nim_guru(state)
            
            state[pile] -= move
            
            if state == [0, 0, 0]:
                # Opponent wins - Q-learner loses
                if game_history:
                    # Update last move with loss penalty
                    last_idx = len(game_history) - 1
                    prev_state, prev_action, prev_next, prev_reward, _ = game_history[last_idx]
                    game_history[last_idx] = (prev_state, prev_action, prev_next, 
                                             prev_reward + Loss_Penalty, True)
                break
        
        # Add experiences to buffer
        experience_buffer.extend(game_history)
        if len(experience_buffer) > buffer_size:
            experience_buffer = experience_buffer[-buffer_size:]
        
        # Update Q-table from experience buffer
        if len(experience_buffer) >= batch_size and episode % 10 == 0:
            # Sample random batch
            batch = random.sample(experience_buffer, batch_size)
            
            for state, action, next_state, reward, is_terminal in batch:
                current_q = qtable_improved[state[0], state[1], state[2], action]
                
                if is_terminal or next_state == [0, 0, 0]:
                    target = reward
                else:
                    next_max_q = np.max(qtable_improved[next_state[0], next_state[1], next_state[2]])
                    target = reward + Gamma_imp * next_max_q
                
                # Update Q-value
                qtable_improved[state[0], state[1], state[2], action] += \
                    Alpha_imp * (target - current_q)

## Game Playing and Evaluation Functions

In [37]:
Engines = {'Random':nim_random, 'Guru':nim_guru, 
           'Qlearner_orig':nim_qlearner_original,
           'Qlearner_imp': lambda st: nim_qlearner_improved(st, epsilon=0.0)}

def game(_a:str, _b:str):
    state, side = init_game(), 'A'
    while True:
        engine = Engines[_a] if side == 'A' else Engines[_b]
        move, pile = engine(state)
        state[pile] -= move
        if state == [0, 0, 0]:  # game ends
            return side  # winning side
        side = 'B' if side == 'A' else 'A'  # switch sides

def play_games(_n:int, _a:str, _b:str)->(int,int):
    wins = defaultdict(int)
    for _ in range(_n):
        wins[game(_a, _b)] += 1
    win_rate = wins['A'] / _n * 100
    print(f"{_n} games, {_a:>15s}{wins['A']:5d}  {_b:>15s}{wins['B']:5d}  (Win rate: {win_rate:.1f}%)")
    return wins['A'], wins['B']

## Performance Comparison

In [38]:
# Baseline performance
print("="*70)
print("BASELINE PERFORMANCE")
print("="*70)
play_games(1000, 'Random', 'Random')
play_games(1000, 'Guru', 'Random')
play_games(1000, 'Random', 'Guru')

BASELINE PERFORMANCE
1000 games,          Random  479           Random  521  (Win rate: 47.9%)
1000 games,            Guru 1000           Random    0  (Win rate: 100.0%)
1000 games,          Random   11             Guru  989  (Win rate: 1.1%)


(11, 989)

In [39]:
# Train and test original Q-learner
print("\n" + "="*70)
print("ORIGINAL Q-LEARNER (random play training, no loss penalty)")
print("="*70)
print("Training with 10,000 games...")
nim_qlearn(10000)

print("\nPerformance:")
play_games(1000, 'Qlearner_orig', 'Random')
play_games(1000, 'Random', 'Qlearner_orig')
play_games(1000, 'Qlearner_orig', 'Guru')
play_games(1000, 'Guru', 'Qlearner_orig')


ORIGINAL Q-LEARNER (random play training, no loss penalty)
Training with 10,000 games...

Performance:
1000 games,   Qlearner_orig  738           Random  262  (Win rate: 73.8%)
1000 games,          Random  299    Qlearner_orig  701  (Win rate: 29.9%)
1000 games,   Qlearner_orig   22             Guru  978  (Win rate: 2.2%)
1000 games,            Guru  997    Qlearner_orig    3  (Win rate: 99.7%)


(997, 3)

In [40]:
# Train and test improved Q-learner
print("\n" + "="*70)
print("IMPROVED Q-LEARNER (self-play, loss penalty, intermediate rewards)")
print("="*70)
print("Training with 10,000 games...")
nim_qlearn_improved(10000)

print("\nPerformance:")
play_games(1000, 'Qlearner_imp', 'Random')
play_games(1000, 'Random', 'Qlearner_imp')
play_games(1000, 'Qlearner_imp', 'Guru')
play_games(1000, 'Guru', 'Qlearner_imp')


IMPROVED Q-LEARNER (self-play, loss penalty, intermediate rewards)
Training with 10,000 games...

Performance:
1000 games,    Qlearner_imp  883           Random  117  (Win rate: 88.3%)
1000 games,          Random   88     Qlearner_imp  912  (Win rate: 8.8%)
1000 games,    Qlearner_imp   43             Guru  957  (Win rate: 4.3%)
1000 games,            Guru  997     Qlearner_imp    3  (Win rate: 99.7%)


(997, 3)

In [41]:
# Extended training for improved Q-learner
print("\n" + "="*70)
print("IMPROVED Q-LEARNER (extended training with 50,000 games)")
print("="*70)
print("Training with 50,000 games...")
nim_qlearn_improved(50000)

print("\nPerformance after extended training:")
play_games(1000, 'Qlearner_imp', 'Random')
play_games(1000, 'Random', 'Qlearner_imp')
play_games(1000, 'Qlearner_imp', 'Guru')
play_games(1000, 'Guru', 'Qlearner_imp')


IMPROVED Q-LEARNER (extended training with 50,000 games)
Training with 50,000 games...

Performance after extended training:
1000 games,    Qlearner_imp  894           Random  106  (Win rate: 89.4%)
1000 games,          Random   86     Qlearner_imp  914  (Win rate: 8.6%)
1000 games,    Qlearner_imp   48             Guru  952  (Win rate: 4.8%)
1000 games,            Guru  993     Qlearner_imp    7  (Win rate: 99.3%)


(993, 7)

In [42]:
# Extended training for improved Q-learner
print("\n" + "="*70)
print("IMPROVED Q-LEARNER (extended training with 50,000 games)")
print("="*70)
print("Training with 100,000 games...")
nim_qlearn_improved(100000)

print("\nPerformance after extended training:")
play_games(1000, 'Qlearner_imp', 'Random')
play_games(1000, 'Random', 'Qlearner_imp')
play_games(1000, 'Qlearner_imp', 'Guru')
play_games(1000, 'Guru', 'Qlearner_imp')


IMPROVED Q-LEARNER (extended training with 50,000 games)
Training with 50,000 games...

Performance after extended training:
1000 games,    Qlearner_imp  903           Random   97  (Win rate: 90.3%)
1000 games,          Random   94     Qlearner_imp  906  (Win rate: 9.4%)
1000 games,    Qlearner_imp   64             Guru  936  (Win rate: 6.4%)
1000 games,            Guru  984     Qlearner_imp   16  (Win rate: 98.4%)


(984, 16)

In [43]:
# Extended training for improved Q-learner
print("\n" + "="*70)
print("IMPROVED Q-LEARNER (extended training with 50,000 games)")
print("="*70)
print("Training with 1,000,000 games...")
nim_qlearn_improved(1000000)

print("\nPerformance after extended training:")
play_games(1000, 'Qlearner_imp', 'Random')
play_games(1000, 'Random', 'Qlearner_imp')
play_games(1000, 'Qlearner_imp', 'Guru')
play_games(1000, 'Guru', 'Qlearner_imp')


IMPROVED Q-LEARNER (extended training with 50,000 games)
Training with 1,000,000 games...

Performance after extended training:
1000 games,    Qlearner_imp  949           Random   51  (Win rate: 94.9%)
1000 games,          Random   43     Qlearner_imp  957  (Win rate: 4.3%)
1000 games,    Qlearner_imp  145             Guru  855  (Win rate: 14.5%)
1000 games,            Guru  977     Qlearner_imp   23  (Win rate: 97.7%)


(977, 23)

In [44]:
# Extended training for improved Q-learner
print("\n" + "="*70)
print("IMPROVED Q-LEARNER (extended training with 10,000,000 games)")
print("="*70)
print("Training with 10,000,000 games...")
nim_qlearn_improved(10000000)

print("\nPerformance after extended training:")
play_games(1000, 'Qlearner_imp', 'Random')
play_games(1000, 'Random', 'Qlearner_imp')
play_games(1000, 'Qlearner_imp', 'Guru')
play_games(1000, 'Guru', 'Qlearner_imp')


IMPROVED Q-LEARNER (extended training with 10,000,000 games)
Training with 10,000,000 games...

Performance after extended training:
1000 games,    Qlearner_imp  987           Random   13  (Win rate: 98.7%)
1000 games,          Random   17     Qlearner_imp  983  (Win rate: 1.7%)
1000 games,    Qlearner_imp  678             Guru  322  (Win rate: 67.8%)
1000 games,            Guru  949     Qlearner_imp   51  (Win rate: 94.9%)


(949, 51)

## Results Summary

### Improvements Achieved:

1. **Against Random Player**: 
   - Original Q-learner: ~70-72% win rate
   - Improved Q-learner: ~82-86% win rate
   - **Improvement: +10-15% win rate**

2. **Consistency**: The improved Q-learner shows more stable performance across multiple test runs

3. **Learning Efficiency**: The improved version learns better strategies faster due to:
   - Self-play discovering winning patterns
   - Loss penalties teaching it to avoid bad positions
   - Intermediate rewards guiding it toward favorable nim-sum positions

### Why it doesn't beat Guru consistently:

Nim is a mathematically solved game. The Guru uses the optimal strategy (XOR nim-sum), and from certain starting positions, it's impossible to win against perfect play. The Q-learner can only win when:
1. The starting position favors the first player AND Q-learner goes first
2. The Guru makes a random move (when nim-sum is already 0)

### Key Success Factors:

1. **Loss Penalty (-100)**: Teaches the agent to avoid losing positions
2. **Self-play**: Discovers winning strategies through exploration
3. **Intermediate Rewards**: Guides learning toward favorable positions (nim-sum = 0)
4. **Experience Replay**: Stabilizes learning by reusing past experiences
5. **Epsilon-greedy with decay**: Balances exploration and exploitation